In [ ]:
# Uncomment the following line to install the required packages

# %pip install langchain openai langchain_openai langchain_core langgraph

In [ ]:
import sys
import os

if 'google.colab' in sys.modules:
    print("Running in Google Colab")
    from google.colab import userdata

    # get secret openai_api_key and set it to OS env OPENAI_API_KEY
    try:
      openai_api_key = userdata.get('openai_api_key')
      os.environ['OPENAI_API_KEY'] = openai_api_key
    except:
      print("No openai_api_key found in Google Colab")

    # get secret openai_base_url
    try:
      openai_base_url = userdata.get('openai_base_url')
      os.environ['OPENAI_API_BASE'] = openai_base_url
    except:
      print("No openai_base_url found in Google Colab")
else:
    print("Not running in Google Colab")

In [ ]:
from typing import Annotated, Sequence, Dict, Any

from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel

import operator
import random

# Can converge correctly

# MODEL_NAME = "anthropic/claude-3.5-sonnet:beta"
# MODEL_NAME = "openai/gpt-4o"
# MODEL_NAME = "openai/gpt-4-turbo"
# MODEL_NAME = "llama3-70b-8192"
# MODEL_NAME = "meta-llama/llama-3-70b-instruct"
# MODEL_NAME = "deepseek/deepseek-chat"
# MODEL_NAME = "qwen/qwen-2-72b-instruct"

# Failed to converge correctly

# MODEL_NAME = "llama3-8b-8192"
# MODEL_NAME = "mistralai/mixtral-8x22b-instruct"
# MODEL_NAME = "anthropic/claude-3-haiku:beta"
MODEL_NAME = "google/gemma-2-9b-it"
# MODEL_NAME = "meta-llama/llama-3-8b-instruct"
# MODEL_NAME = "microsoft/phi-3-medium-128k-instruct"
# MODEL_NAME = "mixtral-8x7b-32768"
# MODEL_NAME = "cohere/command-r"

llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0.5)

# EXECUTOR_MODEL = "microsoft/phi-3-medium-128k-instruct"
# EXECUTOR_MODEL = "deepseek/deepseek-chat"
# EXECUTOR_MODEL = "gemma-7b-it"
# EXECUTOR_MODEL = "llama3-8b-8192"
# EXECUTOR_MODEL = "llama3-70b-8192"
# EXECUTOR_MODEL = "mixtral-8x7b-32768"
# EXECUTOR_MODEL = "anthropic/claude-3-haiku:beta"
# EXECUTOR_MODEL = "meta-llama/llama-3-8b-instruct"
EXECUTOR_MODEL = "google/gemma-2-9b-it"
# EXECUTOR_MODEL = "anthropic/claude-3.5-sonnet:beta"

executor_llm = ChatOpenAI(model_name=EXECUTOR_MODEL, temperature=0.01)

class AgentState(BaseModel):
    # messages: Annotated[Sequence[BaseMessage], operator.add] = []
    acceptance_criteria: str = "Exactly text match."
    user_message: str = ""
    expected_output: str = ""
    system_message: str = ""
    output: str = ""
    suggestions: str = ""
    accepted: bool = False
    analysis: str = ""
    best_output: str = ""
    best_system_message: str = ""
    best_output_age: int = 0
    max_output_age: int = 0

def prompt_developer(state: AgentState) -> AgentState:
    # llm = ChatOpenAI(temperature=0.1)
    
    if not state.system_message:
        # Initial system message creation
        initial_prompt = ChatPromptTemplate.from_messages([
            ("system", """# Expert Prompt Engineer

You are an expert prompt engineer tasked with creating system messages for AI
assistants.

## Instructions

1. Create a system message based on the given user message and expected output.
2. Ensure the system message can handle similar user messages.
3. Output only the system message, without any additional content.
4. Expected Output text should not appear in System Message as an example. But
   it's OK to use some similar text as an example instead.
5. Format the system message well, with no more than 80 characters per line
   (except for raw text).

## Output

Provide only the system message, adhering to the above guidelines.
"""),
            ("human", "User message: {user_message}\nExpected output: {expected_output}\nCreate a system message that will guide the AI to produce the expected output.")
        ])
        response = llm(initial_prompt.format_messages(
            user_message=state.user_message, 
            expected_output=state.expected_output
        ))
        state.system_message = response.content
    else:
        # Update system message based on analysis
        update_prompt = ChatPromptTemplate.from_messages([
            ("system", """# Expert Prompt Engineer

You are an expert prompt engineer tasked with updating system messages for AI
assistants. You Update System Message according to Suggestions, to improve
Output and match Expected Output more closely.

## Instructions

1. Update the system message based on the given Suggestion, User Message, and
   Expected Output.
2. Ensure the updated system message can handle similar user messages.
3. Modify only the content mentioned in the Suggestion. Do not change the
   parts that are not related to the Suggestion.
4. Output only the updated system message, without any additional content.
5. Expected Output text should not appear in System Message as an example. But
   it's OK to use some similar text as an example instead.
   * Remove the Expected Output text or text highly similar to Expected Output
     from System Message, if it's present.
6. Format the system message well, with no more than 80 characters per line
   (except for raw text).

## Output

Provide only the updated System Message, adhering to the above guidelines.
"""),
            ("human", """Current system message: {system_message}
# User Message

{user_message}

# Expected Output

{expected_output}

# Suggestions

{suggestions}
""")
        ])
        response = llm(update_prompt.format_messages(**state.dict()))
        state.system_message = response.content
    print(state.system_message)

    # state.messages.append(SystemMessage(content=state.system_message))
    return state

def prompt_executor(state: AgentState) -> AgentState:
    # llm = ChatOpenAI(temperature=0.1)
    messages = [
        SystemMessage(content=state.system_message),
        HumanMessage(content=state.user_message)
        ]
    response = executor_llm(messages)
    state.output = response.content
    # state.messages.append(HumanMessage(content=state.user_message))
    # state.messages.append(response)

    print(response.content)

    return state

def prompt_analyzer(state: AgentState) -> AgentState:
    # Updated to compare output and expected output with LLM and format the response
    comparison_prompt_template = """
You are a text comparing program. You compare the following output texts and provide a
detailed analysis according to `Acceptance Criteria`. Then you decide whether `Actual Output`
is acceptable.

# Expected Output

```
{expected_output}
```

# Actual Output

```
{output}
```

----

Provide your analysis in the following format:

```
- Acceptable Differences: [List acceptable differences succinctly]
- Unacceptable Differences: [List unacceptable differences succinctly]
- Accept: [Yes/No]
```

* Compare Expected Output and Actual Output with the guidance of Accept Criteria.
* Only set 'Accept' to 'Yes', if Accept Criteria are all met. Otherwise, set 'Accept' to 'No'.
* List only the acceptable differences according to Accept Criteria in 'acceptable Differences' section.
* List only the unacceptable differences according to Accept Criteria in 'Unacceptable Differences' section.

# Acceptance Criteria

```
{acceptance_criteria}
```
"""

    comparison_prompt = ChatPromptTemplate.from_messages([
            ("system", comparison_prompt_template)
        ])
    
    # Format the prompt with the current state
    formatted_prompt = comparison_prompt.format_messages(**state.dict())
    
    # Send the prompt to the LLM
    response = llm(formatted_prompt)
    state.analysis = response.content

    print(response.content)
    
    try:
        # Parse the LLM response to update the state
        analysis_result = parse_llm_response(response.content)
        
        # Update state.matched based on the LLM's analysis
        state.accepted = analysis_result['Accept'].lower() == 'yes'
    except KeyError:
        # If the LLM response is not in the expected format, set matched to False
        state.accepted = False
            
    return state

def parse_llm_response(response: str) -> dict:
    """
    Parses the LLM response to handle both single-line and multi-line formats for Differences and Suggestions.
    """
    lines = response.split('\n')
    result = {}

    # Process each line
    for line in lines:
        # skip the spaces before `- `
        line = line.strip()
        if line.startswith('- Accept:'):
            result['Accept'] = line.split(': ')[1].strip().strip('[]')
            break

    return result

def output_history_analyzer(state: AgentState) -> AgentState:
    system_message_template = """You are a text comparing program. You read the Acceptance Criteria, compare the
compare the exptected output with two different outputs, and decide which one is
more similar to the expected output.

You output the following analysis according to the Acceptance Criteria:

* Your analysis in a Markdown list.
* The ID of the output that is more similar to the Expected Output as Preferred
  Output ID, with the following format:
    
```
# Analysis

...

# Preferred Output ID: [ID]
```

If both outputs are equally similar to the expected output, output the following:

```
# Analysis

...

# Draw
```
"""
    human_message_templates = [
        """
# Output ID: A

```
{best_output}
```

# Output ID: B

```
{output}
```

# Acceptance Criteria

{acceptance_criteria}

# Expected Output

```
{expected_output}
```
""",
        """
# Output ID: B

```
{output}
```

# Output ID: A

```
{best_output}
```

# Acceptance Criteria

{acceptance_criteria}
             
# Expected Output

```
{expected_output}
```
"""
    ]

    # pick a random human message template
    output_comparison_prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_message_template),
        ("human", human_message_templates[random.randint(0, 1)])
        ])

    if (state.best_output is None or state.best_output == "") and \
        (state.best_system_message is None or state.best_system_message == ""):
        state.best_output = state.output
        state.best_system_message = state.system_message
        state.best_output_age = 0

        return state

    response = llm(output_comparison_prompt_template.format_messages(**state.dict()))

    print(response.content)

    result = parse_output_history_analyzer(response.content, 'A')

    if result == 'B':
        state.best_output = state.output
        state.best_system_message = state.system_message
        state.best_output_age = 0
    else:
        state.best_output_age += 1
        state.output = state.best_output
        state.system_message = state.best_system_message

        print("Best Output Age: ", state.best_output_age)

    return state

def parse_output_history_analyzer(response: str, default_result = None) -> dict:
    """
    Parses the LLM response to handle both single-line and multi-line formats for Differences and Suggestions.
    """
    lines = response.split('\n')
    result = default_result

    # Process each line
    for line in lines:
        # skip the spaces before `- `
        line = line.strip()
        if line.startswith('# Preferred Output ID:'):
            result = line.split(': ')[1].strip().strip('[]')
            break
        elif line.startswith('# Draw'):            
            result = default_result
            break

    print("Result: ", result)

    return result

def prompt_suggester(state: AgentState) -> AgentState:
    # Updated to compare output and expected output with LLM and format the response
    suggester_prompt_template = """
Read the following inputs and outputs of an LLM prompt, and also analysis about them.
Then suggest how to improve System Prompt.

System Prompt:
```
{system_message}
```
User Message:
```
{user_message}
```
Expected Output: 
```
{expected_output}
```
Actual Output: 
```
{output}
```

Acceptance Criteria:
```
{acceptance_criteria}
```

Analysis:
```
{analysis}
```

* The goal is to improve the System Prompt to match the Expected Output better.
* Ignore all Acceptable Differences and focus on Unacceptable Differences.
* Suggest formal changes first, then semantic changes.
* Provide your suggestions in a Markdown list, nothing else. Output only the
  suggestions related with Unacceptable Differences.
  * Use `... should ...` to clearly state the desired output.
  * Figue out the contexts of the System Message that conflict with the suggestions,
    and suggest modification or deletion.
* Expected Output text should not appear in System Message as an example. But
  it's OK to use some similar text as an example instead.
  * Ask to remove the Expected Output text or text highly similar to Expected Output
    from System Message, if it's present.
* Provide format examples or detected format name, if System Message does not.
  * Specify the detected format name (e.g. XML, JSON, etc.) of Expected Output, if
    System Message does not mention it.
"""

    suggester_prompt = ChatPromptTemplate.from_messages([
            ("system", suggester_prompt_template)
        ])
    
    # Format the prompt with the current state
    formatted_prompt = suggester_prompt.format_messages(**state.dict())
    
    # Send the prompt to the LLM
    response = llm(formatted_prompt)
    state.suggestions = response.content

    print(response.content)
            
    return state

def should_exit_on_max_age(state: AgentState) -> str:
    if state.max_output_age <=0:
        # always continue if max age is 0
        return "continue"
    
    if state.best_output_age >= state.max_output_age:
        return END
    
    if state.best_output_age > 0:
        # skip prompt_analyzer and prompt_suggester, goto prompt_developer
        return "rerun" 
    
    return "continue"

def should_exit_on_acceptable_output(state: AgentState) -> str:
    if state.accepted:
        return END
    else:
        return "continue"


workflow = StateGraph(AgentState)

workflow.add_node("prompt_developer", prompt_developer)
workflow.add_node("prompt_executor", prompt_executor)
workflow.add_node("output_history_analyzer", output_history_analyzer)
workflow.add_node("prompt_analyzer", prompt_analyzer)
workflow.add_node("prompt_suggester", prompt_suggester)

workflow.set_entry_point("prompt_developer")

workflow.add_edge("prompt_developer", "prompt_executor")
workflow.add_edge("prompt_executor", "output_history_analyzer")

workflow.add_conditional_edges(
    "output_history_analyzer",
    should_exit_on_max_age,
    {
        "continue": "prompt_analyzer",
        "rerun": "prompt_suggester",
        END: END
    }
)

workflow.add_conditional_edges(
    "prompt_analyzer",
    should_exit_on_acceptable_output,
    {
        "continue": "prompt_suggester",
        END: END
    }
)

workflow.add_edge("prompt_suggester", "prompt_developer")

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
initial_states = [
    AgentState(
        max_output_age=3,
        user_message="(2+8)*3",
        expected_output="""(2+8)*3
= 10*3
= 30
""",
        acceptance_criteria="""
* Exactly text match.
* Acceptable differences:
  * Extra or missing spaces.
  * Extra or missing line breaks at the beginning or end of the output.
"""),
    AgentState(
        max_output_age=3,
        user_message="""Here is the GDP data in billions of US dollars (USD) for these years:

Germany:

2015: $3,368.29 billion
2016: $3,467.79 billion
2017: $3,677.83 billion
2018: $3,946.00 billion
2019: $3,845.03 billion
France:

2015: $2,423.47 billion
2016: $2,465.12 billion
2017: $2,582.49 billion
2018: $2,787.86 billion
2019: $2,715.52 billion
United Kingdom:

2015: $2,860.58 billion
2016: $2,650.90 billion
2017: $2,622.43 billion
2018: $2,828.87 billion
2019: $2,829.21 billion
Italy:

2015: $1,815.72 billion
2016: $1,852.50 billion
2017: $1,937.80 billion
2018: $2,073.90 billion
2019: $1,988.14 billion
Spain:

2015: $1,199.74 billion
2016: $1,235.95 billion
2017: $1,313.13 billion
2018: $1,426.19 billion
2019: $1,430.38 billion
""",
        expected_output="""Year,Germany,France,United Kingdom,Italy,Spain
2016-2015,2.96%,1.71%,-7.35%,2.02%,3.04%
2017-2016,5.08%,4.78%,-1.07%,4.61%,6.23%
2018-2017,7.48%,7.99%,7.89%,7.10%,8.58%
2019-2018,-2.56%,-2.59%,0.01%,-4.11%,0.30%
""",
        acceptance_criteria="""
* Strict text matching of the header row and first column(year).
* Acceptable differences:
  * Differences in digital/percentage values in the table, even significant ones.
  * Extra or missing spaces.
  * Extra or missing line breaks.
"""),
    AgentState(
        max_output_age=3,
        user_message="""
Gene sequence: ATGGCCATGGCGCCCAGAACTGAGATCAATAGTACCCGTATTAACGGGTGA
Species: Escherichia coli
""",
        expected_output="""
{
  "Gene Sequence Analysis Results": {
    "Basic Information": {
      "Sequence Length": 54,
      "GC Content": "51.85%"
    },
    "Nucleotide Composition": {
      "A": {"Count": 12, "Percentage": "22.22%"},
      "T": {"Count": 11, "Percentage": "20.37%"},
      "G": {"Count": 16, "Percentage": "29.63%"},
      "C": {"Count": 15, "Percentage": "27.78%"}
    },
    "Codon Analysis": {
      "Start Codon": "ATG",
      "Stop Codon": "TGA",
      "Codon Table": [
        {"Codon": "ATG", "Amino Acid": "Methionine", "Position": 1},
        {"Codon": "GCC", "Amino Acid": "Alanine", "Position": 2},
        {"Codon": "ATG", "Amino Acid": "Methionine", "Position": 3},
        // ... other codons ...
        {"Codon": "TGA", "Amino Acid": "Stop Codon", "Position": 18}
      ]
    },
    "Potential Function Prediction": {
      "Protein Length": 17,
      "Possible Functional Domains": [
        {"Domain Name": "ABC Transporter", "Start Position": 5, "End Position": 15, "Confidence": "75%"},
        {"Domain Name": "Membrane Protein", "Start Position": 1, "End Position": 17, "Confidence": "60%"}
      ],
      "Secondary Structure Prediction": {
        "α-helix": ["2-8", "12-16"],
        "β-sheet": ["9-11"],
        "Random Coil": ["1", "17"]
      }
    },
    "Homology Analysis": {
      "Most Similar Sequences": [
        {
          "Gene Name": "abcT",
          "Species": "Salmonella enterica",
          "Similarity": "89%",
          "E-value": "3e-25"
        },
        {
          "Gene Name": "yojI",
          "Species": "Escherichia coli",
          "Similarity": "95%",
          "E-value": "1e-30"
        }
      ]
    },
    "Mutation Analysis": {
      "SNP Sites": [
        {"Position": 27, "Wild Type": "A", "Mutant": "G", "Amino Acid Change": "Glutamine->Arginine"},
        {"Position": 42, "Wild Type": "C", "Mutant": "T", "Amino Acid Change": "None (Synonymous Mutation)"}
      ]
    }
  }
}
""",
        acceptance_criteria="""
* Consistent with Expected Output:
  * Formats of all JSON sections
  * Data types of all JSON fields
  * Top layer sections
* Acceptable differences:
  * Extra or missing spaces
  * Extra or missing line breaks at the beginning or end of the output
  * Differences in JSON field values
  * JSON wrapped in backquotes
"""),
    AgentState(
        max_output_age=3,
        user_message="""
今天下午3点，在北京国家会议中心，阿里巴巴集团董事局主席马云宣布将投资100亿元人民币用于农村电商发展。这一决定受到了与会代表的热烈欢迎，大家认为这将为中国农村经济带来新的机遇。
""",
        expected_output="""
{
  "文本分析结果": {
    "情感分析": {
      "整体情感": "积极",
      "情感得分": 0.82,
      "情感细分": {
        "乐观": 0.75,
        "兴奋": 0.60,
        "期待": 0.85
      }
    },
    "实体识别": [
      {"实体": "北京", "类型": "地点", "起始位置": 7, "结束位置": 9},
      {"实体": "国家会议中心", "类型": "地点", "起始位置": 9, "结束位置": 15},
      {"实体": "阿里巴巴集团", "类型": "组织", "起始位置": 16, "结束位置": 22},
      {"实体": "马云", "类型": "人物", "起始位置": 26, "结束位置": 28},
      {"实体": "100亿元", "类型": "金额", "起始位置": 32, "结束位置": 37},
      {"实体": "人民币", "类型": "货币", "起始位置": 37, "结束位置": 40},
      {"实体": "中国", "类型": "地点", "起始位置": 71, "结束位置": 73}
    ],
    "关键词提取": [
      {"关键词": "农村电商", "权重": 0.95},
      {"关键词": "马云", "权重": 0.85},
      {"关键词": "投资", "权重": 0.80},
      {"关键词": "阿里巴巴", "权重": 0.75},
      {"关键词": "经济机遇", "权重": 0.70}
    ]
  }
}
""",
        acceptance_criteria="""
* Consistent with Expected Output:
  * Formats of all JSON sections
  * Data types of all JSON fields
  * Top layer sections
* Acceptable differences:
  * Differences in digital values in the table.
  * Extra or missing spaces.
  * Extra or missing line breaks at the beginning or end of the output.
  * Differences in JSON field values
  * Differences in section/item orders.
  * JSON wrapped in backquotes.
"""),
    AgentState(
        max_output_age=3,
        user_message="Low-noise amplifier",
        expected_output="""
A '''low-noise amplifier''' ('''LNA''') is an electronic component that amplifies a very low-power [[signal]] without significantly degrading its [[signal-to-noise ratio]] (SNR). Any [[electronic amplifier]] will increase the power of both the signal and the [[Noise (electronics)|noise]] present at its input, but the amplifier will also introduce some additional noise. LNAs are designed to minimize that additional noise, by choosing special components, operating points, and [[Circuit topology (electrical)|circuit topologies]]. Minimizing additional noise must balance with other design goals such as [[power gain]] and [[impedance matching]].

LNAs are found in [[Radio|radio communications]] systems, [[Amateur Radio]] stations, medical instruments and [[electronic test equipment]]. A typical LNA may supply a power gain of 100 (20&nbsp;[[decibels]] (dB)) while decreasing the SNR by less than a factor of two (a 3&nbsp;dB [[noise figure]] (NF)). Although LNAs are primarily concerned with weak signals that are just above the [[noise floor]], they must also consider the presence of larger signals that cause [[intermodulation distortion]].
""",
        acceptance_criteria="""
* Consistent with Expected Output:
  * Language
  * Text length
  * Text style
  * Text structures
* Cover all the major content of Expected Output.
* Acceptable differences:
  * Minor format differences.
  * Expression differences.
  * Numerical differences.
  * Additional content in Actual Output.
  * Missing minor content in Actual Output.
"""
    ),
    AgentState(
        max_output_age=3,
        user_message="What is the meaning of life?",
        expected_output="""
[
  {"persona": "Philosopher", "prompt": "Explore the concept of life's meaning through the lens of existentialism and purpose-driven existence."},
  {"persona": "Scientist", "prompt": "Examine the biological and evolutionary perspectives on the function and significance of life."},
  {"persona": "Child", "prompt": "Imagine you're explaining to a curious 7-year-old what makes life special and important."}
]
""",
        acceptance_criteria="""
* Consistent with Expected Output:
  * Formats of all JSON sections
  * Data types and formats of all JSON fields
  * Top layer sections
* Acceptable differences:
  * Differences in field values
  * Extra or missing spaces
  * Extra or missing line breaks at the beginning or end of the output
  * JSON wrapped in backquotes
"""
    ),
    AgentState(
        max_output_age=3,
        user_message="""<?php
$username = $_POST['username'];
$password = $_POST['password'];

$query = "SELECT * FROM users WHERE username = '$username' AND password = '$password'";
$result = mysqli_query($connection, $query);

if (mysqli_num_rows($result) > 0) {
    echo "Login successful";
} else {
    echo "Login failed";
}
?>
""",
        expected_output="""
security_analysis:
  vulnerabilities:
    - type: SQL Injection
      severity: Critical
      description: Unsanitized user input directly used in SQL query
      mitigation: Use prepared statements or parameterized queries
    - type: Password Storage
      severity: High
      description: Passwords stored in plain text
      mitigation: Use password hashing (e.g., bcrypt) before storage
  additional_issues:
    - Lack of input validation
    - No CSRF protection
    - Potential for timing attacks in login logic
  overall_risk_score: 9.5/10
  recommended_actions:
    - Implement proper input sanitization
    - Use secure password hashing algorithms
    - Add CSRF tokens to forms
    - Consider using a secure authentication library
""",
        acceptance_criteria="""
* Consistent with Expected Output:
  * Formats of all YAML sections
  * Data types and formats of all YAML fields
  * Top layer sections
* Acceptable differences:
  * Differences in field values
  * Extra or missing spaces
  * Extra or missing line breaks at the beginning or end of the output
  * YAML wrapped in backquotes
"""
    ),
]

selected_states = initial_states[3:4]

for initial_state in selected_states:
    print("User Message:\n", initial_state.user_message)
    print("Expected Output:\n", initial_state.expected_output)

    try:
        config = {"configurable": {"thread_id": "1"}, "recursion_limit": 25}
        result = graph.invoke(initial_state, config)
        print("Final Result:", result)

        # format system message, break it into multiple lines
        print("System Message:")
        print(result['best_system_message'])
        print("Output:")
        print(result['best_output'])
    except Exception as e:
        # print the error message, saying failed to converge
        print("Failed to converge.")
        print(e)

        states = graph.get_state(config)

        # if the length of states is bigger than 0, print the best system message and output
        if len(states) > 0:
            result = states[0]

            print("System Message:")
            print(result['best_system_message'])
            print("Output:")
            print(result['best_output'])